# Machine Learning Project
# Kansas City Crime Data Deep Dive

### Import Dependancies

In [1]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import datetime
import os

### Obtain KCMO crime data (Raw data came from data.kcmo.org)

In [2]:
file_name = os.path.join('Resources', 'KCPD_Crime_Data_2017.csv')
kc_crime = pd.read_csv(file_name)
# kc_crime.Reported_Date = pd.to_datetime(kc_crime.Reported_Date)
kc_crime.reset_index()
kc_crime.head()

,Report_No,Reported_Date,Reported_Time,From_Date,From_Time,To_Date,To_Time,Offense,IBRS,Description,...,Rep_Dist,Area,DVFlag,Invl_No,Involvement,Race,Sex,Age,Firearm Used Flag,Location
0,100080848,5/28/2017,3:44,5/28/2017,2:00,NaN,NaN,1352,280,Stolen Property OFFE,...,PJ3255,CPD,U,1,VIC,W,F,29.0,N,"4000 MILL ST\nKANSAS CITY 64111\n(39.053635, -..."
1,120046817,11/21/2017,13:30,11/20/2017,9:00,NaN,NaN,101,09A,HOMICIDE/Non Neglige,...,PJ1029,CPD,U,1,SUS,B,M,NaN,Y,"1100 LOCUST ST\nKANSAS CITY 64105\n(39.10091, ..."
2,120046817,11/21/2017,13:30,11/20/2017,9:00,NaN,NaN,101,09A,HOMICIDE/Non Neglige,...,PJ1029,CPD,N,1,VIC,B,F,69.0,Y,"1100 LOCUST ST\nKANSAS CITY 64105\n(39.10091, ..."
3,120085080,4/27/2017,11:12,NaN,NaN,NaN,NaN,201,11A,Rape,...,NaN,NaN,U,1,VIC,B,F,21.0,N,99999\n
4,120085080,4/27/2017,11:12,NaN,NaN,NaN,NaN,201,11A,Rape,...,NaN,NaN,U,1,SUS,B,M,52.0,N,99999\n


### Round time to nearest hour

##### The next 3 lines didn't work

In [3]:
# kc_crime_close['time1'] = kc_crime_close['Time'].dt.floor('h')

In [4]:
# kc_crime_close['Time'].dt.round('H').dt.hour

In [5]:
# kc_crime['Reported_Time'] = pd.to_datetime(kc_crime['Reported_Time']) 

### Rename & drop columns

In [6]:
kc_crime.rename(columns={'Zip Code':'zip_code'}, inplace=True)
kc_crime_dropped_columns = kc_crime.drop(['To_Date', 'From_Date', 'To_Time', 'From_Time',
                          'Offense', 'IBRS', 'Rep_Dist', 'Area', 'Beat', 'Address'], axis=1)

### Filter for male and female victims age 90 and under in KCMO

In [7]:
kc_crime_clean_zips = kc_crime_dropped_columns[kc_crime_dropped_columns.zip_code != 99999]
only_kc_crime = kc_crime_clean_zips[kc_crime_clean_zips.City.str.contains("KANSAS CITY") == True]

victims = only_kc_crime[only_kc_crime.Involvement.str.contains("VIC") == True]

victims_no_nans = victims[victims.Sex.str.contains("NaN") == False]
male_female_victims_kcmo = victims_no_nans[victims_no_nans.Sex.str.contains("U") == False]

kc_crime_real_ages = male_female_victims_kcmo[male_female_victims_kcmo['Age'] < 90]

### Copy kc_crime_real_ages and separate "Location" into 3 columns

In [8]:
kc_crime_real_ages_copy = kc_crime_real_ages.copy()
kc_crime_real_ages_copy.head(1)

,Report_No,Reported_Date,Reported_Time,Description,City,zip_code,DVFlag,Invl_No,Involvement,Race,Sex,Age,Firearm Used Flag,Location
0,100080848,5/28/2017,3:44,Stolen Property OFFE,KANSAS CITY,64111,U,1,VIC,W,F,29.0,N,"4000 MILL ST\nKANSAS CITY 64111\n(39.053635, -..."


# Split out geo data

In [9]:
location_only = kc_crime_real_ages_copy['Location'].str[0:-1].str.split('\n', expand=True)
location_only.columns = ("address", "city_zip", "geo")
location_only.head(1)

,address,city_zip,geo
0,4000 MILL ST,KANSAS CITY 64111,"(39.053635, -94.595998"


### Parsing out latitude and longitude

In [10]:
location_only['geo'] = location_only['geo'].str[1:]
geo_split = location_only['geo'].str[0:].str.split(', ', expand=True)
geo_split.columns = ("Latitude", "Longitude")
geo_split.head(1)

,Latitude,Longitude
0,39.053635,-94.595998


In [11]:
kc_crime_real_ages_copy.count()

Report_No            43013
Reported_Date        43013
Reported_Time        43013
Description          43013
City                 43013
zip_code             43013
DVFlag               43013
Invl_No              43013
Involvement          43013
Race                 43013
Sex                  43013
Age                  43013
Firearm Used Flag    43013
Location             43013
dtype: int64

In [12]:
geo_split.count()

Latitude     32267
Longitude    32267
dtype: int64

In [13]:
kcmo_crime_with_nans = pd.concat([kc_crime_real_ages_copy, geo_split], axis=1)
kcmo_crime_with_nans.head(1)

,Report_No,Reported_Date,Reported_Time,Description,City,zip_code,DVFlag,Invl_No,Involvement,Race,Sex,Age,Firearm Used Flag,Location,Latitude,Longitude
0,100080848,5/28/2017,3:44,Stolen Property OFFE,KANSAS CITY,64111,U,1,VIC,W,F,29.0,N,"4000 MILL ST\nKANSAS CITY 64111\n(39.053635, -...",39.053635,-94.595998


In [14]:
kcmo_crime_with_nans.count()

Report_No            43013
Reported_Date        43013
Reported_Time        43013
Description          43013
City                 43013
zip_code             43013
DVFlag               43013
Invl_No              43013
Involvement          43013
Race                 43013
Sex                  43013
Age                  43013
Firearm Used Flag    43013
Location             43013
Latitude             32267
Longitude            32267
dtype: int64

In [15]:
kcmo_crime_no_lat_nans = kcmo_crime_with_nans[kcmo_crime_with_nans.Latitude.str.contains("NaN") == False]
kcmo_crime_no_nans = kcmo_crime_no_lat_nans[kcmo_crime_no_lat_nans.Longitude.str.contains("NaN") == False]
kc_crime_close = kcmo_crime_no_nans.drop(['City', 'DVFlag', 'Invl_No', 'Involvement', 'Firearm Used Flag', 'Location'], axis=1)
kc_crime_close.head(1)

,Report_No,Reported_Date,Reported_Time,Description,zip_code,Race,Sex,Age,Latitude,Longitude
0,100080848,5/28/2017,3:44,Stolen Property OFFE,64111,W,F,29.0,39.053635,-94.595998


In [16]:
kc_crime_close.columns = ("Report", "Date", "Time", "Crime", "Zip", "Race", "Sex", "Age", "Latitude", "Longitude")
kc_crime_close = kc_crime_close.reset_index(drop=True)
kc_crime_close.head()

,Report,Date,Time,Crime,Zip,Race,Sex,Age,Latitude,Longitude
0,100080848,5/28/2017,3:44,Stolen Property OFFE,64111,W,F,29.0,39.053635,-94.595998
1,120046817,11/21/2017,13:30,HOMICIDE/Non Neglige,64105,B,F,69.0,39.10091,-94.577328
2,160028258,8/2/2017,13:50,Auto Theft,64119,W,F,31.0,39.17744,-94.572069
3,160034107,8/24/2017,16:22,Intimidation,64130,W,F,19.0,39.033505,-94.547812
4,160063139,3/15/2017,14:06,Auto Theft,64157,W,F,62.0,39.235881,-94.466171


In [17]:
kc_crime_close.count()

Report       32267
Date         32267
Time         32267
Crime        32267
Zip          32267
Race         32267
Sex          32267
Age          32267
Latitude     32267
Longitude    32267
dtype: int64

### Confirming Clean Data in Excel

In [22]:
kc_crime_close.to_csv('kc_crime_close.csv', index=False)

# Confirm above data is good before proceeding

### Apply get_dummies function

In [ ]:
# Description_columns = pd.get_dummies(kc_crime['Description'],prefix = "Description")
# kc_crime = pd.concat([kc_crime, Description_columns], axis=1)
# kc_crime.drop('Description', axis=1, inplace=True)